# Task_#2_Vysotskiy_Roman_Group_j4132c

## Import libraries

In [405]:
import re
import nltk
from nltk.tokenize import TreebankWordTokenizer, WhitespaceTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize, word_tokenize

# Download necessary methods
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

from collections import Counter

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/roman14/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/roman14/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/roman14/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/roman14/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/roman14/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


## 1. Download Alice in Wonderland by Lewis Carroll from Project Gutenberg's website http://www.gutenberg.org/files/11/11-0.txt
## 2. Perform any necessary preprocessing on the text, including converting to lower case, removing stop words, numbers / non-alphabetic characters, lemmatization.
## 3. Find Top 10 most important (for example, in terms of TF-IDF metric) words from each chapter in the text (not "Alice"); how would you name each chapter according to the identified tokens?
## 4. Find the Top 10 most used verbs in sentences with Alice. What does Alice do most often?

# Part 1

In [57]:
## get text file
# import requests
# url = 'https://www.gutenberg.org/files/11/11-0.txt'
# response = requests.get(url)
#
# with open('11-0.txt', 'wb') as file:
#     file.write(response.content)

In [113]:
# read our text
with open("11-0.txt", encoding='utf-8') as file:
        origin_text = (file.read())
origin_text

'\ufeffCHAPTER I.\nDown the Rabbit-Hole\n\n\nAlice was beginning to get very tired of sitting by her sister on the\nbank, and of having nothing to do: once or twice she had peeped into\nthe book her sister was reading, but it had no pictures or\nconversations in it, “and what is the use of a book,” thought Alice\n“without pictures or conversations?”\n\nSo she was considering in her own mind (as well as she could, for the\nhot day made her feel very sleepy and stupid), whether the pleasure of\nmaking a daisy-chain would be worth the trouble of getting up and\npicking the daisies, when suddenly a White Rabbit with pink eyes ran\nclose by her.\n\nThere was nothing so _very_ remarkable in that; nor did Alice think it\nso _very_ much out of the way to hear the Rabbit say to itself, “Oh\ndear! Oh dear! I shall be late!” (when she thought it over afterwards,\nit occurred to her that she ought to have wondered at this, but at the\ntime it all seemed quite natural); but when the Rabbit actually

In [130]:
origin_text = re.sub(r"\ufeff", "", origin_text)
origin_text

'CHAPTER I.\nDown the Rabbit-Hole\n\n\nAlice was beginning to get very tired of sitting by her sister on the\nbank, and of having nothing to do: once or twice she had peeped into\nthe book her sister was reading, but it had no pictures or\nconversations in it, “and what is the use of a book,” thought Alice\n“without pictures or conversations?”\n\nSo she was considering in her own mind (as well as she could, for the\nhot day made her feel very sleepy and stupid), whether the pleasure of\nmaking a daisy-chain would be worth the trouble of getting up and\npicking the daisies, when suddenly a White Rabbit with pink eyes ran\nclose by her.\n\nThere was nothing so _very_ remarkable in that; nor did Alice think it\nso _very_ much out of the way to hear the Rabbit say to itself, “Oh\ndear! Oh dear! I shall be late!” (when she thought it over afterwards,\nit occurred to her that she ought to have wondered at this, but at the\ntime it all seemed quite natural); but when the Rabbit actually _took

### As we can see our text has a lot of punctuations and other symbols. Let's preprocess it.

# Part 2

### Preprocessing, converting to lower case, removing stop words, numbers / non-alphabetic characters, lemmatization.

In [471]:
def punctuation_clear(text):
    import string
    for p in string.punctuation + '\n':
        if p in text:
            text = text.replace(p, ' ')
    return text

def preprocessing_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r"([^\w\n\s])", "", text, re.UNICODE)
    text = text.lower()
    text = ' '.join([word for word in text.split() if not word in stop_words])
    text = [lemmatizer.lemmatize(token) for token in text.split(' ')]
    text = " ".join(text)
    return text

In [311]:
text = punctuation_clear(origin_text)
text

'CHAPTER I  Down the Rabbit Hole   Alice was beginning to get very tired of sitting by her sister on the bank  and of having nothing to do  once or twice she had peeped into the book her sister was reading  but it had no pictures or conversations in it  “and what is the use of a book ” thought Alice “without pictures or conversations ”  So she was considering in her own mind  as well as she could  for the hot day made her feel very sleepy and stupid   whether the pleasure of making a daisy chain would be worth the trouble of getting up and picking the daisies  when suddenly a White Rabbit with pink eyes ran close by her   There was nothing so  very  remarkable in that  nor did Alice think it so  very  much out of the way to hear the Rabbit say to itself  “Oh dear  Oh dear  I shall be late ”  when she thought it over afterwards  it occurred to her that she ought to have wondered at this  but at the time it all seemed quite natural   but when the Rabbit actually  took a watch out of its 

In [312]:
text = re.split(r'CHAPTER ', text)[1:]
text

['I  Down the Rabbit Hole   Alice was beginning to get very tired of sitting by her sister on the bank  and of having nothing to do  once or twice she had peeped into the book her sister was reading  but it had no pictures or conversations in it  “and what is the use of a book ” thought Alice “without pictures or conversations ”  So she was considering in her own mind  as well as she could  for the hot day made her feel very sleepy and stupid   whether the pleasure of making a daisy chain would be worth the trouble of getting up and picking the daisies  when suddenly a White Rabbit with pink eyes ran close by her   There was nothing so  very  remarkable in that  nor did Alice think it so  very  much out of the way to hear the Rabbit say to itself  “Oh dear  Oh dear  I shall be late ”  when she thought it over afterwards  it occurred to her that she ought to have wondered at this  but at the time it all seemed quite natural   but when the Rabbit actually  took a watch out of its waistco

### Let's make some more preprocessing like Lemmatization/Steamming and removing stopwords.
### Lemmatization and stemming are special cases of normalization, and they are different.
### Stemming is a rough heuristic process that cuts off the "superfluous" from the root of words, often this leads to the loss of word–forming suffixes.
### Lemmatization is a more subtle process that uses vocabulary and morphological analysis to eventually bring a word to its canonical form – a lemma.

In [313]:
preprocessed_text = [preprocessing_text(chapter) for chapter in text]
preprocessed_text

['rabbit hole alice beginning get tired sitting sister bank nothing twice peeped book sister reading picture conversation use book thought alice without picture conversation considering mind well could hot day made feel sleepy stupid whether pleasure making daisy chain would worth trouble getting picking daisy suddenly white rabbit pink eye ran close nothing remarkable alice think much way hear rabbit say oh dear oh dear shall late thought afterwards occurred ought wondered time seemed quite natural rabbit actually took watch waistcoat pocket looked hurried alice started foot flashed across mind never seen rabbit either waistcoat pocket watch take burning curiosity ran across field fortunately time see pop large rabbit hole hedge another moment went alice never considering world get rabbit hole went straight like tunnel way dipped suddenly suddenly alice moment think stopping found falling deep well either well deep fell slowly plenty time went look wonder going happen next first tried

### We see that our text has been changed - all words with lower case, stopwords like 'was' have been removed e.t.c.
### Now we need to count the words in each chapter.
### Frequency scoring has a problem: the words with the highest frequency have, respectively, the highest score. There may not be as much informational gain for the model in these words as in less frequent words. One way to remedy the situation is to lower the rating of a word that is often found in all similar documents. This is called TF-IDF.

### TF-IDF (short for term frequency — inverse document frequency) is a statistical measure to assess the importance of a word in a document that is part of a collection or corpus.

### TF-IDF scoring increases in proportion to the frequency of occurrence of a word in a document, but this is offset by the number of documents containing this word.

# Part 3

In [434]:
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(preprocessed_text) # compute tf-idf scoring for each sentence
feature_names = vectorizer.get_feature_names_out()
print(X.shape)

(12, 2384)


### So we got 12 (chapters) rows and 2384 columns (words)

In [483]:
for i, chapter in enumerate(X):
    tfidf_scores = X[i].toarray()[0]
    indices = tfidf_scores.argsort()[::-1]
    top_word = [feature_names[j] for j in indices if feature_names[j] != 'alice'][:12] # chose 12 due to equal counts
    print(f"Chapter {i+1}, Top 10 most important words: {', '.join(top_word)}")

Chapter 1, Top 10 most important words: little, bat, door, rabbit, key, way, eat, like, think, either, see, bottle
Chapter 2, Top 10 most important words: mouse, pool, little, oh, swam, cat, dear, said, foot, mabel, go, cried
Chapter 3, Top 10 most important words: said, mouse, dodo, race, lory, prize, dry, thimble, know, bird, dinah, course
Chapter 4, Top 10 most important words: bill, rabbit, little, window, puppy, one, chimney, glove, fan, bottle, said, room
Chapter 5, Top 10 most important words: caterpillar, said, serpent, pigeon, youth, egg, you, size, father, little, well, hookah
Chapter 6, Top 10 most important words: said, footman, cat, baby, mad, duchess, wow, like, pig, cook, little, much
Chapter 7, Top 10 most important words: hatter, dormouse, said, march, hare, tea, twinkle, time, well, treacle, draw, clock
Chapter 8, Top 10 most important words: queen, said, hedgehog, king, gardener, soldier, cat, rose, five, procession, executioner, head
Chapter 9, Top 10 most important

### Now let's imagine on chapters names:
#### Chapter 1 - Little rabbit and key from the door
#### Chapter 2 - Pool of cryes
#### Chapter 3 - Race of glory
#### Chapter 4 - Meeting of rabbit
#### Chapter 5 - Speaking caterpillar
#### Chapter 6 - Footman, duchess, pig
#### Chapter 7 - Sweetie tea with hatter
#### Chapter 8 - Procession with a Queen of Rose
#### Chapter 9 - Moral story of queen and duchess
#### Chapter 10 - The Dinner
#### Chapter 11 - Court with jurors and 'Bread with butter'
#### Chapter 12 - Power in the justice

### Original chapter names:
#### 'CHAPTER I.     Down the Rabbit-Hole',
#### ' CHAPTER II.    The Pool of Tears',
#### ' CHAPTER III.   A Caucus-Race and a Long Tale',
#### ' CHAPTER IV.    The Rabbit Sends in a Little Bill',
#### ' CHAPTER V.     Advice from a Caterpillar',
#### ' CHAPTER VI.    Pig and Pepper',
#### ' CHAPTER VII.   A Mad Tea-Party',
#### ' CHAPTER VIII.  The Queen’s Croquet-Ground',
#### ' CHAPTER IX.    The Mock Turtle’s Story',
#### ' CHAPTER X.     The Lobster Quadrille',
#### ' CHAPTER XI.    Who Stole the Tarts?',
#### ' CHAPTER XII.   Alice’s Evidence',

# Part 4

In [472]:
sentences_verbs = preprocessing_text(origin_text)
sentences_verbs = sent_tokenize(sentences_verbs)
sentences_verbs

['chapter rabbithole alice beginning get tired sitting sister bank nothing twice peeped book sister reading picture conversation use book thought alice without picture conversation considering mind well could hot day made feel sleepy stupid whether pleasure making daisychain would worth trouble getting picking daisy suddenly white rabbit pink eye ran close nothing _very_ remarkable alice think _very_ much way hear rabbit say oh dear oh dear shall late thought afterwards occurred ought wondered time seemed quite natural rabbit actually _took watch waistcoatpocket_, looked it, hurried on, alice started feet, flashed across mind never seen rabbit either waistcoat-pocket, watch take it, burning curiosity, ran across field it, fortunately time see pop large rabbit-hole hedge.',
 'another moment went alice it, never considering world get again.',
 'rabbit-hole went straight like tunnel way, dipped suddenly down, suddenly alice moment think stopping found falling deep well.',
 'either well de

In [485]:
tagged_words = []
for sentence in range(len(sentences_verbs)):
  tagged = nltk.tag.pos_tag(sentences_verbs[sentence].split(' '), tagset='universal')
  # if ('alice', 'NN') or ('she', 'PRP$') or ('alice', 'NNP') in tagged:
  if ('alice', 'NOUN') in tagged: # it might be better to add condition "or ('she', 'PRON')" but "she" might use with other (f.e. Queen)
    tagged_words.append(tagged)
tagged_words

[[('chapter', 'NOUN'),
  ('rabbithole', 'ADJ'),
  ('alice', 'NOUN'),
  ('beginning', 'VERB'),
  ('get', 'VERB'),
  ('tired', 'ADJ'),
  ('sitting', 'VERB'),
  ('sister', 'ADJ'),
  ('bank', 'NOUN'),
  ('nothing', 'NOUN'),
  ('twice', 'ADV'),
  ('peeped', 'VERB'),
  ('book', 'NOUN'),
  ('sister', 'NOUN'),
  ('reading', 'VERB'),
  ('picture', 'NOUN'),
  ('conversation', 'NOUN'),
  ('use', 'ADP'),
  ('book', 'NOUN'),
  ('thought', 'NOUN'),
  ('alice', 'NOUN'),
  ('without', 'ADP'),
  ('picture', 'NOUN'),
  ('conversation', 'NOUN'),
  ('considering', 'VERB'),
  ('mind', 'NOUN'),
  ('well', 'NOUN'),
  ('could', 'VERB'),
  ('hot', 'VERB'),
  ('day', 'NOUN'),
  ('made', 'VERB'),
  ('feel', 'NOUN'),
  ('sleepy', 'ADJ'),
  ('stupid', 'ADJ'),
  ('whether', 'ADP'),
  ('pleasure', 'NOUN'),
  ('making', 'NOUN'),
  ('daisychain', 'NOUN'),
  ('would', 'VERB'),
  ('worth', 'VERB'),
  ('trouble', 'NOUN'),
  ('getting', 'VERB'),
  ('picking', 'VERB'),
  ('daisy', 'NOUN'),
  ('suddenly', 'ADV'),
  ('white'

In [498]:
# Count all verbs in tagged sentences
verbs = [verb for sentence in tagged_words for verb, tag in sentence if tag.endswith('VERB')]
sorted(Counter(verbs).items(), key=lambda item: (-item[1], item[0]))[:11]

[('said', 219),
 ('’', 121),
 ('“', 84),
 ('would', 34),
 ('went', 33),
 ('could', 30),
 ('know', 28),
 ('looked', 26),
 ('say', 25),
 ('thought', 23),
 ('go', 22)]

### Usually Alice talked, went, thought, got smth, looked, assumed (might) e.t.c.
### She was so active

In [402]:
lemmatizer.lemmatize('alice beginning get tired sitting sister bank nothing twice peeped book sister reading picture conversation use book thought alice without picture conversation considering mind well could hot day made feel')

'alice beginning get tired sitting sister bank nothing twice peeped book sister reading picture conversation use book thought alice without picture conversation considering mind well could hot day made feel'

### I show to you cell above that lemmatization doens't work correct. As I understood it should change the form of verb, but it didn't.